In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Train.xlsx')

In [3]:
train.head(5)

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [4]:
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\niles\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\niles\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
STORY      7628 non-null object
SECTION    7628 non-null int64
dtypes: int64(1), object(1)
memory usage: 119.3+ KB


In [6]:
train.shape

(7628, 2)

In [7]:
train.groupby("SECTION").describe()

STORY                                                               
        count unique                                                top freq
SECTION                                                                     
0        1686   1673  This story has been published from a wire agen...    4
1        2772   2731  This story has been published from a wire agen...   13
2        1924   1914  We will leave no stone unturned to make the au...    3
3        1246   1233  This story has been published from a wire agen...   11

In [8]:
train.drop_duplicates(inplace = True)

In [9]:
all_punctuation = string.punctuation + '‘’,:”][],'

In [10]:
def punc_remover(raw_text):
    no_punct = "".join([i for i in raw_text if i not in all_punctuation])
    return no_punct

In [11]:
def stopword_remover(no_punct_text):
    words = no_punct_text.split()
    no_stop_word = " ".join([i for i in words if i not in stopwords.words('english')])
    return no_stop_word

In [12]:
lemmer = nltk.stem.WordNetLemmatizer()
def lem(words):
    return " ".join([lemmer.lemmatize(words,'v') for word in words.split()]) 

In [13]:
def text_cleaner(raw):
    cleaned_text = stopword_remover(punc_remover(raw))
    return lem(cleaned_text)

In [14]:
text_cleaner("Hi!, this is a sample text to test the text cleaner method. Removes *@!#special characters%$^* and stopwords. And lemmatizes, go, going - run, ran, running")

'Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special characters stopwords And lemmatizes go going run ran running Hi sample text test text cleaner method Removes special charact

In [15]:
train['CLEAN_STORY'] = train['STORY'].apply(text_cleaner)

In [16]:
print(train.values)

[['But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront.\n\n\nGill’s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.\n\n\nBut there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor.'
  3
  'But painful huge reversal fee income unheard among private sector lenders Essentially means Yes Bank took granted fees structured loan deal

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
bow_dict = CountVectorizer().fit(train["CLEAN_STORY"])
len(bow_dict.vocabulary_)

39336

In [20]:
bow_dict = bow_dict.transform(train["CLEAN_STORY"])

In [21]:
print(bow_dict.shape)

(7551, 39336)


In [27]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_Transformer = TfidfTransformer().fit(bow_dict)

In [28]:
storytfidf = tfidf_Transformer.transform(bow_dict)

In [29]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(storytfidf,train["SECTION"])